# Добавление истории сообщений

Как правило пользователь хочет обмениваться сообщениями с вопросно-ответными приложениями.
В таком случае приложению нужна «память» о предыдущих вопросах и ответах и логика для включения этих данных в текущие рассуждения.

В этом разделе показано как добавить историю сообщений в приложение, разработанное в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart).

:::note

Подробнее об истории сообщений — в разделе [Работа с историей сообщений](/docs/expression_language/how_to/message_history).

:::

Для добавления истории сообщений в приложение потребуется:

1. Доработать промпт, чтобы он поддерживал историю сообщений на входе.
2. Добавить цепочку, которая берет последний вопрос пользователя и переформулирует его в контексте истории чата. Это нужно, если последний вопрос ссылается на контекст предыдущих сообщений. Например, если пользователь задает уточняющий вопрос: «Расскажи подробнее о втором пункте?». На такой вопрос нельзя ответить без предыдущего контекста.

## Установка зависимостей

Для разработки используются модели генерации и эмбеддингов GigaChat, а также векторное хранилище FAISS.
Вы можете использовать любую [векторное хранилище](/docs/modules/data_connection/vectorstores/) или [ретривер](/docs/modules/data_connection/retrievers/)

Установите пакеты с помощью команды:

In [ ]:
%pip install --upgrade --quiet  gigachain langchainhub faiss-cpu bs4

## Пример цепочки без истории сообщений

Пример цепочки без поддержки истории сообщений, разобранный в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart):

In [2]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# Загрузка, разделение на части и индексация содержимого блога.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents=splits, embedding=GigaChatEmbeddings(credentials="<авторизационные_данные>", verify_ssl_certs=False))

# Извлечение данных и генерация с помощью релевантных фрагментов блога.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [4]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complicated task into smaller and simpler steps. This allows an agent to better understand and plan ahead for the task. CoT (Chain of Thought) and Tree of Thoughts are techniques that help enhance model performance on complex tasks by decomposing them into manageable steps. Task decomposition can be done using LLMs with simple prompting, task-specific instructions, or with human inputs.'

## Добавление контекста к вопросу

Сначала нужно определить цепочку, которая:

* получает на вход историю сообщений и последний вопрос пользователя;
* переформулирует вопрос, если он ссылается на данные, доступные в истории сообщений.

Добавим промпт, который содержит переменную `MessagesPlaceholder` под названием `chat_history`.
С помощью ключа `chat_history` в промпт можно передать историю сообщений, которая будет помещена после системного промпта и перед последним вопросом пользователя.

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

Полученная цепочка переформулирует вопрос, который ссылается на предыдущие данные, и, таким образом позволяет обращаться к истории сообщений:

In [8]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

'The term "large" in this context refers to the size of the model, which is typically much larger than standard language models.'

## Цепочка с историей чата

Теперь можно собрать итоговую вопросно-ответную логику.

В ней цепочка `contextualize_q_chain` выполняет только при наличии истории сообщений.
Это возможно благодаря тому, что если функция в LCEL-цепочке возвращает другую цепочку, она также будет вызвана.

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [10]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

AIMessage(content='Common ways of doing Task Decomposition include using Chain of Thought (CoT), which instructs the model to think step by step and utilize more test-time computation to decompose hard tasks into smaller and simpler steps.', response_metadata={'token_usage': Usage(prompt_tokens=703, completion_tokens=49, total_tokens=752), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'})

Настоящее вопросно-ответное приложение должно уметь сохранять историю сообщений, обновлять ее и использовать в контексте.

Такую функциональность можно добавить с помощью классов:

- [BaseChatMessageHistory](/docs/modules/memory/chat_messages/) — хранение истории сообщений.
- [RunnableWithMessageHistory](/docs/expression_language/how_to/message_history) — обертка для LCEL-цепочки и `BaseChatMessageHistory`, которая преобразует историю сообщений в вводные данные и обновляет ее после каждого вызова.

Подробно о том как использовать эти классы для разработки разговорной цепочки — в разделе [Работа с историей сообщений](/docs/expression_language/how_to/message_history).
